In [62]:
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker


import sys 
import os
path = os.path.dirname(os.getcwd())
sys.path.append(path)
from lib import *

In [63]:
# Results Import
optimization_result = pd.read_csv("../results/1_optimization_result.csv")
baseline_result = pd.read_csv("../results/2_baseline.csv")
simpleCA_result = pd.read_csv("../results/3_simpleCA_result.csv")
randomizedEB_result = pd.read_csv("../results/4_randomEB_optimization.csv")
leftover_result = pd.read_csv("../results/5_leftover_optimization_result.csv")
leftover_resultWeighted = pd.read_csv("../results/7_leftoverWeighted_optimization_result.csv")

leftoverAddToEB_result = pd.read_csv("../results/6_leftoverEB_optimization_result.csv")
op_baseline_result = pd.read_csv("../results/8_baselineOptimization_result.csv")

randomizedEB_result_08_12 = pd.read_csv("../results/9_randomEB_optimization_08_12.csv")

# Data Import
df = pd.read_csv(r'../data/DE_2020.csv')
ci_data_2020 = df['carbon_intensity_avg']
df = pd.read_csv(r'../data/DE_2021.csv')
ci_data_2021 = df['carbon_intensity_avg']
df = pd.read_csv(r'../data/projectcount_wikiDE_2014.csv')
clickData_hourly_2014 = df["de"]
df = pd.read_csv(r'../data/projectcount_wikiDE_2015.csv')
clickData_hourly_2015 = df["De"].tolist()
clickData_hourly_2015 = clickData_hourly_2015[24:] + clickData_hourly_2015[0:24]

carbonBudget = calcCarbonBudgetHourInWeekAVG(2020,clickData_hourly_2014,ci_data_2020)

In [64]:
ed_avg_2021 = calcEnergyDemandFromAVG(clickData_hourly_2015)

ce_avg_2021 = calcCarbonEmissionFromEnergyDemand(ed_avg_2021,ci_data_2021)

q_avg_2021 = calcQFromAVG(clickData_hourly_2015)

sum_q_avg_2021 = sum([q_avg_2021[i][-1] for i in range(len(q_avg_2021))])

print("CE-Increase compared to AVG: " + str((sum(optimization_result["ce"])- sum(ce_avg_2021)) / sum(ce_avg_2021) * 100))
print("Q compared to AVG: " + str((sum(optimization_result["user-throughput"])- sum_q_avg_2021) / sum_q_avg_2021 * 100))

CE-Increase compared to AVG: 9.627935370297537
Q compared to AVG: 38.17042484441831


# Carbon Budget Utilization

In [65]:
weekday_frequency = weekdayfrequency(2021)
s = weekday_frequency.index(53)*24
optimal_cb = []
simple_cb = []
baseline_lp_cb = []
baseline_np_cb = []
baseline_hp_cb = []
randomEB_cb = []
leftover_result_cb = []
leftover_resultWeighted_cb = []
op_baseline_result_cb = []
leftoverAddToEB_result_cb = []
randomizedEB_result_08_12_cb = []
#op_cb = []
for i in range(8760):
    index = (i+s) % 168
    optimal_cb.append(optimization_result['ce'][i] / carbonBudget[index])
    simple_cb.append(simpleCA_result['ce'][i]  / carbonBudget[index])
    baseline_lp_cb.append((baseline_result['lp'][i]*ci_data_2021[i]) / carbonBudget[index])
    baseline_np_cb.append((baseline_result['np'][i]*ci_data_2021[i]) / carbonBudget[index])
    baseline_hp_cb.append((baseline_result['hp'][i]*ci_data_2021[i]) / carbonBudget[index])
    randomEB_cb.append(randomizedEB_result['ce'][i]  / carbonBudget[index])
    leftover_result_cb.append(leftover_result['ce'][i]  / carbonBudget[index])
    leftover_resultWeighted_cb.append(leftover_resultWeighted['ce'][i]  / carbonBudget[index])
    op_baseline_result_cb.append(op_baseline_result['ce'][i]  / carbonBudget[index])
    leftoverAddToEB_result_cb.append(leftoverAddToEB_result['ce'][i]  / carbonBudget[index])
    randomizedEB_result_08_12_cb.append(randomizedEB_result_08_12['ce'][i] / carbonBudget[index])

In [66]:
print("Mean CB Utilization")
print("Optimal: ", np.mean(optimal_cb)*100)
print("Simple: ", np.mean(simple_cb)*100)
print("Baseline LP: ", np.mean(baseline_lp_cb)*100)
print("Baseline NP: ", np.mean(baseline_np_cb)*100)
print("Baseline HP: ", np.mean(baseline_hp_cb)*100)
print("Random EB (0.8-1.2) ", np.mean(randomizedEB_result_08_12_cb)*100)
print("Random EB (1.0-1.4): ", np.mean(randomEB_cb)*100)
print("Leftover: ", np.mean(leftover_result_cb)*100)
print("Leftover Weighted: ", np.mean(leftover_resultWeighted_cb)*100)
print("Leftover Hour-to-Hour: ", np.mean(leftoverAddToEB_result_cb)*100)
print("Optimum: ", np.mean(op_baseline_result_cb)*100)

Mean CB Utilization
Optimal:  85.15942326104079
Simple:  58.740198835693434
Baseline LP:  11.36080618455312
Baseline NP:  133.3677408378274
Baseline HP:  343.2683756058953
Random EB (0.8-1.2)  85.11098954962752
Random EB (1.0-1.4):  99.0634093498923
Leftover:  105.27125220725824
Leftover Weighted:  99.60398878215297
Leftover Hour-to-Hour:  94.17004073587934
Optimum:  100.78777741308717


In [68]:
print("Mean Revenue (in q)")
print("Optimal: ", np.mean(optimization_result['q'])*100)
print("Simple: ", np.mean(simpleCA_result['q'])*100)
print("Random EB (0.8-1.2) ", np.mean(randomizedEB_result_08_12['q'])*100)
print("Random EB (1.0-1.4) ", np.mean(randomizedEB_result['q'])*100)
print("Leftover: ", np.mean(leftover_result['q'])*100)
print("Leftover Weighted: ", np.mean(leftover_resultWeighted['q'])*100)
print("Leftover Hour-to-Hour: ", np.mean(leftoverAddToEB_result["q"])*100)
print("Optimum: ", np.mean(op_baseline_result['q'])*100)

Mean Revenue (in q)
Optimal:  58.86844713184981
Simple:  49.60349568493198
Random EB (0.8-1.2)  58.59677957619711
Random EB (1.0-1.4)  61.1006128741441
Leftover:  61.772084640413574
Leftover Weighted:  60.86443998287878
Leftover Hour-to-Hour:  59.89959005137124
Optimum:  77.3216106164334


In [42]:
ed = calcEnergyDemandFromAVG(clickData_hourly_2014)
carbonEmission = calcCarbonEmissionFromEnergyDemand(ed,ci_data_2020)
baseline_lp_ce = []
baseline_np_ce = []
baseline_hp_ce = []

for i in range(8760):
    baseline_lp_ce.append(baseline_result['lp'][i]*ci_data_2021[i])
    baseline_np_ce.append(baseline_result['np'][i]*ci_data_2021[i])
    baseline_hp_ce.append(baseline_result['hp'][i]*ci_data_2021[i])

print("Distance to goal in %")
print("Optimal: " + str((sum(carbonEmission)-sum(optimization_result['ce']))/sum(carbonEmission)*100 -7.7))
print("Simple: " + str((sum(carbonEmission)-sum(simpleCA_result['ce']))/sum(carbonEmission)*100 -7.7))
print("Baseline LP: " + str((sum(carbonEmission)-sum(baseline_lp_ce))/sum(carbonEmission)*100 -7.7))
print("Baseline NP: " + str((sum(carbonEmission)-sum(baseline_np_ce))/sum(carbonEmission)*100 -7.7))
print("Baseline HP: " + str((sum(carbonEmission)-sum(baseline_hp_ce))/sum(carbonEmission)*100 -7.7))
print("Random EB (1.0-1.4) " + str((sum(carbonEmission)-sum(randomizedEB_result['ce']))/sum(carbonEmission)*100 -7.7))
print("Leftover: " + str((sum(carbonEmission)-sum(leftover_result['ce']))/sum(carbonEmission)*100 -7.7))
print("Leftover Weighted: " + str((sum(carbonEmission)-sum(leftover_resultWeighted['ce']))/sum(carbonEmission)*100 -7.7))
print("Leftover Hour-to-Hour: " + str((sum(carbonEmission)-sum(leftoverAddToEB_result['ce']))/sum(carbonEmission)*100 -7.7))
print("Optimum: " + str((sum(carbonEmission)-sum(op_baseline_result['ce']))/sum(carbonEmission)*100 -7.7))

Distance to goal in %
Optimal: 14.134021484540053
Simple: 38.28152716754724
Baseline LP: 82.04118577391074
Baseline NP: -28.610619033482585
Baseline HP: -219.14217526198198
Random EB: 1.2634120163061793
Leftover: 0.6991152461706678
Leftover Weighted: 0.6493013186342926
Leftover Hour-to-Hour: 5.883501958073951
Optimum: 0.00015873023349843862


In [43]:
ci_list = ci_data_2020.tolist()
ci_data = ci_list[0:1416] + ci_list[1416+24:8784]
revenue_2020 = calcQFromAVG(clickData_hourly_2014)

baseline_lp_Q = []
baseline_np_Q = []
baseline_hp_Q = []

for i in range(8760):
    baseline_lp_Q.append(clickData_hourly_2015[i]*0.1670625)
    baseline_np_Q.append(clickData_hourly_2015[i]*0.30071250000000005)
    baseline_hp_Q.append(clickData_hourly_2015[i]*0.8464499999999999)

ut_2020 = 0
for i in range(len(revenue_2020)):
    ut_2020 += revenue_2020[i][-1]

In [44]:
print("Distance revenue in user-throughput")
print("Optimal: " + str((sum(optimization_result['user-throughput'])-ut_2020)/ut_2020*100))
print("Simple: " + str((sum(simpleCA_result['user-throughput'])-ut_2020)/ut_2020*100))
print("Baseline LP: " + str((sum(baseline_lp_Q)-ut_2020)/ut_2020*100))
print("Baseline NP: " + str((sum(baseline_np_Q)-ut_2020)/ut_2020*100))
print("Baseline HP: " + str((sum(baseline_hp_Q)-ut_2020)/ut_2020*100))
print("Random EB (1.0-1.4):" + str((sum(randomizedEB_result['user-throughput'])-ut_2020)/ut_2020*100))
print("Leftover: " + str((sum(leftover_result['user-throughput'])-ut_2020)/ut_2020*100))
print("Leftover Weighted: " + str((sum(leftover_resultWeighted['user-throughput'])-ut_2020)/ut_2020*100))
print("Leftover Hour-to-Hour: " + str((sum(leftoverAddToEB_result['user-throughput'])-ut_2020)/ut_2020*100))
print("Optimum: " + str((sum(op_baseline_result['user-throughput'])-ut_2020)/ut_2020*100))

Distance revenue in user-throughput
Optimal: 17.050538280616994
Simple: -2.4912362957158973
Baseline LP: -66.43259003768551
Baseline NP: -39.57866206783386
Baseline HP: 70.07487714239404
Random EB: 21.66637756959042
Leftover: 21.730599393746523
Leftover Weighted: 20.964160099164168
Leftover Hour-to-Hour: 18.987176810171345
Optimum: 56.195726159600554


In [45]:
print("CE Reduction ")
print("Optimal: " + str((sum(carbonEmission)-sum(optimization_result['ce']))/sum(carbonEmission)*100))
print("Simple: " + str((sum(carbonEmission)-sum(simpleCA_result['ce']))/sum(carbonEmission)*100))
print("Baseline LP: " + str((sum(carbonEmission)-sum(baseline_lp_ce))/sum(carbonEmission)*100))
print("Baseline NP: " + str((sum(carbonEmission)-sum(baseline_np_ce))/sum(carbonEmission)*100))
print("Baseline HP: " + str((sum(carbonEmission)-sum(baseline_hp_ce))/sum(carbonEmission)*100))
print("Random EB (1.0-1.4): " + str((sum(carbonEmission)-sum(randomizedEB_result['ce']))/sum(carbonEmission)*100))
print("Leftover: " + str((sum(carbonEmission)-sum(leftover_result['ce']))/sum(carbonEmission)*100))
print("Leftover Weighted: " + str((sum(carbonEmission)-sum(leftover_resultWeighted['ce']))/sum(carbonEmission)*100))
print("Leftover Hour-to-Hour: " + str((sum(carbonEmission)-sum(leftoverAddToEB_result['ce']))/sum(carbonEmission)*100))
print("Optimum: " + str((sum(carbonEmission)-sum(op_baseline_result['ce']))/sum(carbonEmission)*100))

CE Reduction 
Optimal: 21.834021484540052
Simple: 45.98152716754724
Baseline LP: 89.74118577391074
Baseline NP: -20.910619033482586
Baseline HP: -211.442175261982
Random EB: 8.96341201630618
Leftover: 8.399115246170668
Leftover Weighted: 8.349301318634293
Leftover Hour-to-Hour: 13.583501958073951
Optimum: 7.700158730233499


In [46]:
print("Difference Basline N to Simple: " + str((sum(simpleCA_result['user-throughput'])-sum(baseline_np_Q))/sum(simpleCA_result['user-throughput'])*100))

Difference Basline N to Simple: 38.034966666784335


# Carbon Budget Utilization

In [47]:
df = pd.read_csv(r'../data/projectcount_wikiDE_2014.csv')
clickData_hourly_2014 = df["de"]
carbonBudget = calcCarbonBudgetHourInWeekAVG(2020,clickData_hourly_2014,ci_data_2020)
optimal_eb = []
simple_eb = []
baseline_lp_cb = []
baseline_np_cb = []
baseline_hp_cb = []
op_cb = []
leftover_result_eb = []
leftover_resultWeighted_ed = []
randomEB_result_eb = []
randomEB_075_result_eb = []

year=2015
weekday_frequency = weekdayfrequency(year)
s = weekday_frequency.index(53)*24

In [48]:
for i in range(0,8760):
    optimal_eb.append(carbonBudget[(i+s) % 168] - optimization_result['ed'][i])
    simple_eb.append(carbonBudget[(i+s) % 168] - simpleCA_result['ed'][i])
    leftover_result_eb.append(carbonBudget[(i+s) % 168])
    randomEB_result_eb.append(carbonBudget[(i+s) % 168] - randomizedEB_result['ed'][i])
    #randomEB_075_result_eb.append(randomEB_075_result['ed'][i]  / carbonBudget[(i+s) % 168])

In [49]:
EB_annum = 109090108.87360902

print("Gap to budget (total): ", sum(optimal_eb))
print("Gap to budget (in percent): ", sum(optimization_result['ed']) / EB_annum *100) 
print("Gap to budget (in percent): ", (1 -(sum(optimization_result['ed']) / EB_annum)) *100) 

Gap to budget (total):  37440134751.763725
Gap to budget (in percent):  98.10377998980229
Gap to budget (in percent):  1.8962200101977023


# Exceeding Budget Approaches

In [50]:
print("Recycle: Gap to budget (total): ", EB_annum - sum(leftover_result["ed"]))
print("Recycle: Gap to budget (in %): ", sum(leftover_result["ed"]) / EB_annum *100)
print("")
print("Weighted Recycle: Gap to budget (total): ", EB_annum - sum(leftover_resultWeighted["ed"]))
print("Weighted Recycle: Gap to budget (in %): ", sum(leftover_resultWeighted["ed"]) / EB_annum *100)
print("")
print("Randomiized Result: Gap to budget (total): ", EB_annum - sum(randomizedEB_result["ed"]))
print("Randomiized Result: Gap to budget (in %): ", sum(randomizedEB_result["ed"]) / EB_annum *100)
#print("")
#print("Randomiized Result: Gap to budget (total): ", EB_annum - sum(randomEB_075_result["ed"]))
#print("Randomiized Result: Gap to budget (in %): ", sum(randomizedEB_result["ed"]) / EB_annum *100)

Recycle: Gap to budget (total):  -17375363.626391545
Recycle: Gap to budget (in %):  115.92753349116418

Weighted Recycle: Gap to budget (total):  -17374495.82639207
Weighted Recycle: Gap to budget (in %):  115.92673800199616

Randomiized Result: Gap to budget (total):  -16528328.19639133
Randomiized Result: Gap to budget (in %):  115.15107865144853


# Total annual carbon emissions

In [51]:
# Carbon Budget calculation
carbonEmission_2020 = calcCarbonEmissionFromEnergyDemand(calcEnergyDemandFromAVG(clickData_hourly_2014),ci_data)
carbonEmission_2020 = sum(carbonEmission_2020) # / 8784)* 8760 # Leap Year

#carbonBudget_anually = sum(calcCarbonEmissionFromEnergyDemand(calcEnergyDemandFromAVG()))
carbonBudget_anually = carbonEmission_2020 * 0.92

In [52]:
optimalCE = sum([optimization_result["ed"][i] * ci_data_2021[i] for i in range(0,8760)])
simpleCA_resultCE = sum([simpleCA_result["ed"][i] * ci_data_2021[i] for i in range(0,8760)])
leftover_resultCE = sum([leftover_result["ed"][i] * ci_data_2021[i] for i in range(0,8760)])
leftover_resultWeightedCE = sum([leftover_resultWeighted["ed"][i] * ci_data_2021[i] for i in range(0,8760)])
randomEB_result = sum([randomizedEB_result["ed"][i] * ci_data_2021[i] for i in range(0,8760)])

In [53]:
print("Optimal: Carbon Budget gap (total): ", carbonBudget_anually - optimalCE)
print("Optimal: Carbon Budget gap (in %): ", (optimalCE / carbonBudget_anually) *100)
print("")
print("Simple: Carbon Budget gap (total): ", carbonBudget_anually - simpleCA_resultCE)
print("Simple: Carbon Budget gap (in %): ", (simpleCA_resultCE / carbonBudget_anually) *100)
print("")
print("Recycle: Carbon Budget gap (total): ", carbonBudget_anually - leftover_resultCE)
print("Recycle: Carbon Budget gap (in %): ", (leftover_resultCE / carbonBudget_anually) *100)
print("")
print("Weighted Recycle: Carbon Budget gap (total): ", carbonBudget_anually - leftover_resultWeightedCE)
print("Weighted Recycle: Carbon Budget gap (in %): ", (leftover_resultWeightedCE / carbonBudget_anually) *100)
print("")
print("Randomized Result (1.0-1.4): Carbon Budget gap (total): ", carbonBudget_anually - randomEB_result)
print("Randomized Result (1.0-1.4): Carbon Budget gap (in %): ", (randomEB_result / carbonBudget_anually) *100)


Optimal: Carbon Budget gap (total):  5644221768.169201
Optimal: Carbon Budget gap (in %):  84.96302012549994

Simple: Carbon Budget gap (total):  15496301105.71622
Simple: Carbon Budget gap (in %):  58.71573133962256

Recycle: Carbon Budget gap (total):  162837318.34320068
Recycle: Carbon Budget gap (in %):  99.56617908024927

Weighted Recycle: Carbon Budget gap (total):  142513448.34827423
Weighted Recycle: Carbon Budget gap (in %):  99.62032465365837

Randomized Result: Carbon Budget gap (total):  393067994.0195236
Randomized Result: Carbon Budget gap (in %):  98.95281302575415
